In [1]:
import os
os.chdir('../')
os.getcwd()

'e:\\github_clone\\GridFormer-14'

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from gridformer.Utils.dataset import load_observation, ObsDataset, DataLoader

In [3]:
class AutoEncoder(nn.Module):
    def __init__(self, config) -> None:
        super(AutoEncoder, self).__init__()
        self.config = config
        input_dim = self.config.input_dim
        latent_dim = self.config.latent_dim

        self.encoder = nn.Sequential(
                        nn.Linear(input_dim, input_dim//2),
                        nn.ReLU(),
                        nn.Linear(input_dim//2, input_dim//4),
                        nn.ReLU(),
                        nn.Linear(input_dim//4, input_dim//8),
                        nn.ReLU(),
                        nn.Linear(input_dim//8, self.config.latent_dim)
        )

        self.decoder = nn.Sequential(
                        nn.Linear(latent_dim, latent_dim*2),
                        nn.ReLU(),
                        nn.Linear(latent_dim*2, latent_dim*4),
                        nn.ReLU(),
                        nn.Linear(latent_dim*4, latent_dim*8),
                        nn.ReLU(),
                        nn.Linear(latent_dim*8, input_dim)
        )

        self.optimizer = optim.Adam(self.parameters(), lr=0.00004)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(device=self.device)


    def forward(self, x):
        z = self.encoder(x)
        pred_x = self.decoder(z)
        return z, pred_x
    

    def save_model(self):
        torch.save(self.state_dict(), os.path.join(self.config.path, "autoEncoder.pt"))
        print(f"model saved at {self.config.path}")

    def load_model(self):
        self.load_state_dict(torch.load(os.path.join(self.config.path, "autoEncoder.pt")))
        print(f"model loaded at {self.config.path}")

    def train_data(self, dataloader, data_variance, save_path=None):
        
        best_score = float('inf')
        num = 0
        for observation in dataloader:
            num += 1
            z, pred_x = self.forward(observation)
            loss = F.mse_loss(pred_x, observation) #/ data_variance

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
                


            print(
                f"Epoch {num} - "
                f"Reconstruction Loss: {loss:.4f}, "
            ) 

            if loss < best_score:
                self.save_model()
                best_score = loss

    
    def evaluate(self, dataloader, data_variance):
        self.eval()
        total_loss = 0
        num = 0
        with torch.no_grad():
            for observation in dataloader:
                num += 1
                z, pred_x = self.forward(observation)
                loss = F.mse_loss(pred_x, observation) #/ data_variance
                    
                total_loss += loss.item()

        print(
            f"Epoch {num} - "
            f"Reconstruction Loss: {(total_loss / num):.4f}, "
        )

In [4]:
class Config:
    input_dim = 467
    latent_dim = 64
    path = "gridformer\\models"

In [5]:
config = Config()
ae = AutoEncoder(config)

In [6]:
obs = load_observation("C:\\Users\\Ernest\\Downloads\\topo_data", start=0, end=5)
dataset = ObsDataset(obs, device=ae.device)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
data_variance = np.var(obs)

In [13]:
ae.train_data(dataloader=dataloader, data_variance=data_variance)

Epoch 1 - Reconstruction Loss: 1090.2736, 
model saved at gridformer\models
Epoch 2 - Reconstruction Loss: 1154.6630, 
Epoch 3 - Reconstruction Loss: 1841.3090, 
Epoch 4 - Reconstruction Loss: 1380.3938, 
Epoch 5 - Reconstruction Loss: 1538.9746, 
Epoch 6 - Reconstruction Loss: 1750.6334, 
Epoch 7 - Reconstruction Loss: 1174.0406, 
Epoch 8 - Reconstruction Loss: 943.7601, 
model saved at gridformer\models
Epoch 9 - Reconstruction Loss: 1757.3833, 
Epoch 10 - Reconstruction Loss: 1701.3608, 
Epoch 11 - Reconstruction Loss: 1241.8396, 
Epoch 12 - Reconstruction Loss: 1592.6758, 
Epoch 13 - Reconstruction Loss: 1390.4172, 
Epoch 14 - Reconstruction Loss: 1142.2469, 
Epoch 15 - Reconstruction Loss: 920.0459, 
model saved at gridformer\models
Epoch 16 - Reconstruction Loss: 1679.9768, 
Epoch 17 - Reconstruction Loss: 1286.1874, 
Epoch 18 - Reconstruction Loss: 1357.3896, 
Epoch 19 - Reconstruction Loss: 2058.0747, 
Epoch 20 - Reconstruction Loss: 2316.5647, 
Epoch 21 - Reconstruction Loss: 

In [14]:
ae.load_model()

model loaded at gridformer\models


In [15]:
ae.evaluate(dataloader=dataloader, data_variance=data_variance)

Epoch 1599 - Reconstruction Loss: 1362.6445, 


In [16]:
obs_t = torch.tensor(np.array(obs[0], np.float32), dtype=torch.float32, device=ae.device)

In [17]:
z, pred_x = ae.forward(obs_t.unsqueeze(0))

In [18]:
F.mse_loss(obs_t, pred_x)

C:\Users\Ernest\AppData\Local\Temp\ipykernel_6668\1498174816.py:1: UserWarning: Using a target size (torch.Size([1, 467])) that is different to the input size (torch.Size([467])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  F.mse_loss(obs_t, pred_x)


tensor(134.5717, device='cuda:0', grad_fn=<MseLossBackward0>)

In [1]:
import torch

def discretize_continuous_values(z_continuous, num_categories):
    """
    Convert continuous values into discrete categorical values using fixed bins.

    Args:
        z_continuous (Tensor): Continuous latent variables of shape (batch_size, latent_dim).
        num_categories (int): Number of categories (bins) for each latent dimension.

    Returns:
        Tensor: Categorical latent variables of shape (batch_size, latent_dim), with discrete integer indices.
    """
    # Get the min and max range for the continuous values (assuming normalized range [-1, 1])
    z_min, z_max = -1.0, 1.0  # You can customize this range based on your encoder's output

    # Scale the continuous values to the range [0, num_categories)
    z_scaled = (z_continuous - z_min) / (z_max - z_min) * num_categories

    # Discretize by assigning to the nearest integer bucket
    z_discrete = torch.clamp(z_scaled.long(), 0, num_categories - 1)  # Ensure values are within valid category range

    return z_discrete


In [3]:

# Example Usage
latent_dim = 64  # Latent space size
num_categories = 64  # Number of categories per latent dimension

# Continuous encoder output
z_continuous = torch.randn(1, latent_dim)  # Example batch size 16

# Discretize
z_discrete = discretize_continuous_values(z_continuous, num_categories)

print("Discrete Latent Shape:", z_discrete.shape)  # (16, 64)
print("Discrete Latent Values:", z_discrete)

Discrete Latent Shape: torch.Size([1, 64])
Discrete Latent Values: tensor([[35,  0, 63,  0, 63, 52,  0, 63, 63, 53, 51,  3, 11, 23,  3, 26, 63, 52,
         32, 55, 60, 63, 63, 31, 10,  4, 23, 63, 21, 20, 63, 30, 27, 59, 63, 23,
         20, 32, 63,  5, 56, 22, 27,  9,  0, 49, 28, 63,  0, 32,  0, 63, 51, 27,
         21,  0, 46, 43, 32, 13, 61, 43, 47,  0]])
